In [1]:
import pandas as pd
df = pd.read_csv("tweets.csv",encoding='ISO-8859-1')
df.head(5)

,ï»¿id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [2]:
df = df.drop(['ï»¿id','keyword','location'],axis=1)
df.head()

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0


In [3]:
df['target'].value_counts()

0    9256
1    2114
Name: target, dtype: int64

In [4]:
df_0_class = df[df['target']==0]
df_1_class = df[df['target']==1]
df_0_class_undersampled = df_0_class.sample(df_1_class.shape[0])
df = pd.concat([df_0_class_undersampled, df_1_class], axis=0)


In [5]:
df['target'].value_counts()

0    2114
1    2114
Name: target, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['target'], stratify=df['target'])

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [8]:
import os
os.environ["TFHUB_CACHE_DIR"] = "some_dir"
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
bert_preprocess_model = hub.KerasLayer(preprocess)
bert_encoder = hub.KerasLayer(encoder)

In [9]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text-layer')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_encoder(preprocessed_text)
d_layer = tf.keras.layers.Dropout(0.1, name="dropout-layer")(outputs['pooled_output'])
d_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(d_layer)
model = tf.keras.Model(inputs=[text_input], outputs = [d_layer])

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text-layer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text-layer[0][0]']             
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [11]:
m= [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=m)

In [12]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
100/100 [==============================] - 788s 8s/step - loss: 0.6343 - accuracy: 0.6455 - precision: 0.6451 - recall: 0.6467
Epoch 2/10
100/100 [==============================] - 866s 9s/step - loss: 0.5953 - accuracy: 0.6966 - precision: 0.6970 - recall: 0.6953
Epoch 3/10
100/100 [==============================] - 748s 7s/step - loss: 0.5735 - accuracy: 0.7133 - precision: 0.7175 - recall: 0.7035
Epoch 4/10
100/100 [==============================] - 874s 9s/step - loss: 0.5638 - accuracy: 0.7256 - precision: 0.7372 - recall: 0.7009
Epoch 5/10
100/100 [==============================] - 818s 8s/step - loss: 0.5521 - accuracy: 0.7319 - precision: 0.7351 - recall: 0.7249
Epoch 6/10
100/100 [==============================] - 753s 8s/step - loss: 0.5391 - accuracy: 0.7376 - precision: 0.7418 - recall: 0.7287
Epoch 7/10
100/100 [==============================] - 714s 7s/step - loss: 0.5356 - accuracy: 0.7468 - precision: 0.7529 - recall: 0.7344
Epoch 8/10
100/100 [==============

In [13]:
model.evaluate(X_test, y_test)

34/34 [==============================] - 254s 7s/step - loss: 0.5187 - accuracy: 0.7512 - precision: 0.8023 - recall: 0.6673


[0.5187166929244995,
 0.7511826157569885,
 0.8022727370262146,
 0.6672967672348022]

In [14]:
import numpy as np
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_predicted)
matrix

array([[441,  87],
       [176, 353]], dtype=int64)

In [15]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.71      0.84      0.77       528
           1       0.80      0.67      0.73       529

    accuracy                           0.75      1057
   macro avg       0.76      0.75      0.75      1057
weighted avg       0.76      0.75      0.75      1057



In [16]:
X_trialTest = [
    "500 people died from a collapsed building",
    "Eruption in Indonesia",
    "I Am a disco dancer",
    "Will she ever be mine?",
    "I am the monster",
    "My heart is flooded",
     "Fisherman recalls racing tsunami, raining rocks after Tongan volcano erupted",
    "I am melting for her",
    "I took a hit",
    "Road safety panel concerned over rise in accidents on highways",
    "Maharashtra: NGT nails CSTPS for high pollution, slaps Rs 5 crore",
    "No effective provision to punish, fine nylon manja ban violators: HC",
    "2,498 farmers ended their lives in 11 months of 2021 in Maharashtra: RTI reply",
    "Delhi: One-stop portal to drive electric vehicle journey",
    "Tongan 'real life Aquaman' survives 27-hour swim after tsunami",
    "Biden administration unveils plan to combat worsening US wildfires",
    "Karol Bagh parking: AAP, BJP spar over tendering process",
    "Bulli Bai case: Mumbai police arrest MBA graduate from Odisha",
    "Trees damaged on CR Park stretch again: Activist",
    "Magnitude 5.6 quake hits western Afghanistan, killing more than 20"
    
]

In [17]:
# y_predicted = model.predict(X_trialTest)
# print(y_predicted)
# counter = 0
# for i in y_predicted:
#     print(X_trialTest[counter]+"-------->", "True" if i >0.75 else "False")
#     counter+=1

In [19]:
model.save("model",include_optimizer=False) 

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets
